# Part 01
### Optimize SVM Parameters (C and Gamma) on RBF Kernel

In [1]:
# imports
import random
import numpy
import math
import operator

from deap import base, creator, tools, algorithms, benchmarks

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_boston
from sklearn import metrics

import matplotlib.pyplot as plt

In [2]:
db = load_boston()

In [3]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness, speed=list, smin=0.0, smax=1.0, best=1.0)
creator.create("Particle", list, fitness=creator.Fitness, speed=list, smin=None, smax=None, best=None)

In [4]:
def generate_particle(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

In [5]:
toolbox = base.Toolbox()

c_min = math.pow(2, -15)
c_max = math.pow(2, 3)
g_min = math.pow(2, -5)
g_max = math.pow(2, 15)

pmin = -100
pmax = 100

toolbox.register("c", random.uniform, c_min, c_max)
toolbox.register("gamma", random.uniform, g_min, g_max)

# used for GA
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.c, toolbox.gamma), 1)
toolbox.register("population_ga", tools.initRepeat, list, toolbox.individual)

# used for PSO
toolbox.register("particle", generate_particle, size=1, pmin=pmin, pmax=pmax, smin=pmin*0.1, smax=pmax*0.1)
toolbox.register("population_pso", tools.initRepeat, list, toolbox.particle)

In [6]:
def evaluate_ga(individual):
    # X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)
    X = db.data
    y = db.target
    
    clf = SVR(kernel='rbf', C=individual[0], gamma=individual[1])
    
    scores = cross_val_score(clf, X, y, cv=5)
    acc = scores.mean()
    
    return acc,

In [7]:
def custom_mutation(ind):
    return ind,

In [8]:
def update_particle(part, best, c, gamma):
    X = db.data
    y = db.target
    
    clf = SVR(kernel='rbf', C=c, gamma=gamma)
    
    scores = cross_val_score(clf, X, y, cv=5)
    acc = scores.mean()
    
    part[:] = [acc]

In [9]:
toolbox.register("evaluate_ga", evaluate_ga)
toolbox.register("mutate", custom_mutation)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("select", tools.selTournament, tournsize=3)

toolbox.register("update", update_particle, c=)
toolbox.register("evaluate_pso", evaluate_pso)

In [10]:
def apply_ga():
    ngen = 100
    # mu = 200
    # lam = 20
    cxpb = 0.7
    mutpb = 0.2
    
    pop = toolbox.population_ga(n=10)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, 
                                       stats=stats, halloffame=hof, verbose=False)
    
    return pop, log, hof
    
def apply_pso():
    pop = toolbox.population_pso(n=10)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    log = tools.Logbook()
    log.header = ["gen", "evals"] + stats.fields
    GEN = 100
    best = None
    
    all_bests = []

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate_pso(part)
            
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
                
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        
        all_bests.append(best.fitness.values)
        
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best, all_bests

In [11]:
if __name__ == "__main__":
#     results_ga = apply_ga()
#     print(results_ga[1])
    results_pso = apply_pso()
    print(results_pso[1])

[-95.65728245722316]


IndexError: list index out of range